In [ ]:
import astra
import numpy as np
import scipy.io 
import skimage.io as io
import matplotlib.pyplot as plt

%matplotlib inline

Select if CUDA shall be used or not

In [ ]:
def recon(sino,angles, Niterations=200,center=0,cuda=True) :
    if cuda == True :
        projstr='cuda'
        cfgstr='SIRT_CUDA'
    else :
        projstr='strip'
        cfgstr='SIRT'
        
    vol_geom = astra.create_vol_geom(sino.shape[1],sino.shape[1])
    proj_geom = astra.create_proj_geom('parallel', 1, sino.shape[1], angles)
    proj_geom_cor = astra.geom_postalignment(proj_geom,center)
    sino_id = astra.data2d.link("-sino",proj_geom_cor,sino) # share sinogram array with ASTRA
    proj_id = astra.create_projector(projstr, proj_geom_cor, vol_geom)
    # Create a data object for the reconstruction
    rec_id = astra.data2d.create('-vol', vol_geom)
    
    cfg = astra.astra_dict(cfgstr)
    cfg['ReconstructionDataId'] = rec_id
    cfg['ProjectionDataId']     = sino_id
    cfg['ProjectorId']          = proj_id
    cfg['option'] = {}
    cfg['option']['MinConstraint'] = 0.  # Force solution to be nonnegative.

    # Create the algorithm object from the configuration structure
    alg_id = astra.algorithm.create(cfg)
    astra.algorithm.run(alg_id, Niterations)
    
    rec = astra.data2d.get(rec_id)
    
    return rec

In [ ]:
def showRecon(sino,rec,l0=0,l1=0) :
    plt.figure(figsize=[15,5])
    
    plt.subplot(1,3,1)
    plt.imshow(sino,cmap='viridis')
    plt.title("Sinogram")
    if (l0!=l1) :
        plt.plot([0,sino.shape[1]-1],[l0,l0],'r')
        plt.plot([0,sino.shape[1]-1],[l1,l1],'r')
    plt.subplot(1,3,2)
    plt.imshow(rec,cmap='viridis')
    plt.title('Reconstruction')
  #  plt.colorbar()
    plt.subplot(1,3,3)
    h,a=np.histogram(np.squeeze(rec),bins=100)
    plt.plot(a[1:-1],h[1:])
    plt.title('Histogram')

In [ ]:
def normImage(img,ob,dc) :
    ob2= ob-dc
    ob2[ob2<=0] = 1
    norm = (img - dc)/ob2
    
    return norm

## Oblique projections (in-plane/through-plane)

The reconstruction tests are centered around these two projections.

In [ ]:
ob = io.imread("../data/ob_0001.tif")
dc = io.imread("../data/dc_0001.tif")
pip = io.imread("../data/fctomo_0281.tif")
ptp = io.imread("../data/fctomo_0564.tif")
nip = normImage(pip,ob,dc)
ntp = normImage(ptp,ob,dc)
plt.figure(figsize=[15,6])
plt.subplot(1,2,1)
plt.imshow(nip, clim=[0,1.2])
plt.plot([0,nip.shape[1]-1],[445,445],'r')
plt.title('Inplane projection')
plt.subplot(1,2,2)
plt.imshow(ntp, clim=[0,1.2])
plt.plot([0,nip.shape[1]-1],[445,445],'r');
plt.title('Through plane projection');


# Load sinogram

The sinogram corresponds to line 445 of the projections

In [ ]:
sino=io.imread("../data/FCSino_0445.tif")
plt.imshow(sino)
angles = np.linspace(0,2*np.pi,sino.shape[0],False)

## Reconstruction of full sinogram
All lines in the sinogram are reconstructed as comparison for the following tests with missing wedge.

In [ ]:
rec=recon(sino,angles,center=-29)

In [ ]:
showRecon(sino,rec)

## Sinogram angles

In [ ]:
center_ip = 281 # In-plane
center_tp = 564 # Through plane
dTheta    = 180*(angles[1]-angles[0])/np.pi # angle step in degrees
wedge     = 15 # degrees

In [ ]:
20/dTheta

## Missing wedge tests in-plane
The in-plane tests select a $\pm$15 degrees wedge with membrane parallel to the beam.

In [ ]:
idxw  = np.ceil(wedge/dTheta).astype(int)
sip   = sino[center_ip-idxw:center_ip+idxw,:]
aip   = angles[center_ip-idxw:center_ip+idxw]
plt.imshow(sip)

In [ ]:
recip_sym=recon(sip,aip,center=-29)
showRecon(sino,recip_sym,center_ip-idxw,center_ip+idxw)

The in-plane test is able to identify planes parallel to the beam. The water droplet bundle is located but it is hard to separate the droplets from each other.

## Asymmetric wedge in-plane
The in-plane tests select a $0$-30 degrees wedge with membrane parallel to the beam.

In [ ]:
idxw  = np.ceil(wedge/dTheta).astype(int)
sip   = sino[center_ip:center_ip+2*idxw,:]
aip   = angles[center_ip:center_ip+2*idxw]
plt.imshow(sip)

In [ ]:
recip_asym=recon(sip,aip,center=-29)
showRecon(sino,recip_asym,center_tp,center_tp+2*idxw)

On first sight, there is a better separation beween the droplets with the asymmetric wedge. This is because there is greater tilt of the sample in one direction that allows the view between the droplets.

### Comparison in-plane close-up

In [ ]:
roi=[1000,1200,1200,1700]
plt.figure(figsize=[12,4])
plt.subplot(1,3,1)
plt.imshow(rec[roi[0]:roi[1],roi[2]:roi[3]])
plt.title("Full reconstruction")
plt.subplot(1,3,2)
plt.imshow(recip_sym[roi[0]:roi[1],roi[2]:roi[3]])
plt.title("$\pm${0} deg wedge reconstruction".format(wedge))

plt.subplot(1,3,3)
plt.imshow(recip_asym[roi[0]:roi[1],roi[2]:roi[3]])
plt.title("0-{0} deg wedge reconstruction".format(2*wedge))

# Missing wedge test through plane
In this test the central beam is perpendicular to the membrane. The selected wedge is again $\pm$15 degrees from the perpendicular beam.

In [ ]:
idxw  = np.ceil(wedge/dTheta).astype(int)
stp   = sino[center_tp-idxw:center_tp+idxw,:]
atp   = angles[center_tp-idxw:center_tp+idxw]
plt.imshow(stp)

## Reconstruction of through plane wedge

In [ ]:
rectp_sym=recon(stp,atp,center=-29)
showRecon(sino,rectp_sym,center_tp-idxw,center_tp+idxw)

## Asymmetric wedge through plane

In [ ]:
idxw  = np.ceil(wedge/dTheta).astype(int)
stp   = sino[center_tp:center_tp+2*idxw,:]
atp   = angles[center_tp:center_tp+2*idxw]
plt.imshow(stp)

In [ ]:
rectp_asym=recon(stp,atp,center=-29)
showRecon(sino,rectp_asym,center_tp-idxw,center_tp+idxw)

In [ ]:
roi=[1000,1200,1200,1700]
plt.figure(figsize=[12,4])
plt.subplot(1,3,1)
plt.imshow(rec[roi[0]:roi[1],roi[2]:roi[3]])
plt.title("Full reconstruction")
plt.subplot(1,3,2)
plt.imshow(rectp_sym[roi[0]:roi[1],roi[2]:roi[3]])
plt.title("$\pm${0} deg wedge reconstruction".format(wedge))

plt.subplot(1,3,3)
plt.imshow(rectp_asym[roi[0]:roi[1],roi[2]:roi[3]])
plt.title("0-{0} deg wedge reconstruction".format(2*wedge))

In [ ]:
plt.figure(figsize=[12,8])
plt.subplot(1,3,1)
plt.imshow(rectp_sym)
plt.title('Through-plane missing wedge $\pm${0}'.format(wedge))

plt.subplot(1,3,2)
plt.imshow(rec)
plt.title('Full reconstruction')

plt.subplot(1,3,3)
plt.imshow(recip_sym)
plt.title('In-plane missing wedge $\pm${0}'.format(wedge))

